In [ ]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 14.7 MB/s 


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
import os, shutil
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
original_dataset_dir = '/content/drive/MyDrive/data/'
base_dir = '/content/drive/MyDrive/data/'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
conv_base = tf.keras.applications.EfficientNetB4(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3))

71686520/71686520 [==============================] - 3s 0us/step


In [ ]:
conv_base.summary()

Model: "efficientnetb4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 tf.math.truediv (TFOpLambda)   (None, 224, 224, 3)  0           ['normalization[0][0

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb4 (Functional)  (None, 7, 7, 1792)       17673823  
                                                                 
 flatten (Flatten)           (None, 87808)             0         
                                                                 
 dense (Dense)               (None, 1024)              89916416  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 8)                 8200      
                                                                 
Total params: 107,598,439
Trainable params: 107,473,232
Non-trainable params: 125,207
_________________________________________________________________


In [ ]:
conv_base.trainable = False

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

batch = 50

train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    shuffle=True)
label_map = (train_generator.class_indices)

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(256, 256),
        batch_size=batch,
        class_mode='categorical',
        shuffle=True)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=batch,
    class_mode='categorical',
    shuffle=False)

label_map

Found 5599 images belonging to 8 classes.
Found 1600 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


{'Засветы': 0,
 'Малая часть упаковки видна на фото': 1,
 'Не читабилен текст': 2,
 'Нормальные': 3,
 'Размытые': 4,
 'Фото не в том режиме': 5,
 'Фото полки где 4-5 товара и видно что фоткали не основ товар': 6,
 'Фото ценника': 7}

In [ ]:
train_stop = train_generator.samples//batch
validation_stop = validation_generator.samples//batch
test_stop = test_generator.samples//batch

In [ ]:
from tensorflow.keras import optimizers

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc',
                       tf.keras.metrics.Recall(),
                       tf.keras.metrics.Precision(),
                       tfa.metrics.F1Score(num_classes=8),
                       tfa.metrics.FBetaScore(num_classes=8)])

In [ ]:
%%time
history = model.fit(train_generator,
                    steps_per_epoch=train_stop,
                    epochs=25)

Epoch 1/25
111/111 [==============================] - 2362s 21s/step - loss: 0.9549 - acc: 0.7403 - recall: 0.7151 - precision: 0.7773 - f1_score: 0.7377 - fbeta_score: 0.7377
Epoch 2/25
111/111 [==============================] - 324s 3s/step - loss: 0.4655 - acc: 0.8564 - recall: 0.8436 - precision: 0.8728 - f1_score: 0.8556 - fbeta_score: 0.8556
Epoch 3/25
111/111 [==============================] - 326s 3s/step - loss: 0.3862 - acc: 0.8807 - recall: 0.8728 - precision: 0.8919 - f1_score: 0.8803 - fbeta_score: 0.8803
Epoch 4/25
111/111 [==============================] - 327s 3s/step - loss: 0.3278 - acc: 0.8976 - recall: 0.8919 - precision: 0.9054 - f1_score: 0.8975 - fbeta_score: 0.8975
Epoch 5/25
111/111 [==============================] - 316s 3s/step - loss: 0.2811 - acc: 0.9113 - recall: 0.9047 - precision: 0.9177 - f1_score: 0.9110 - fbeta_score: 0.9110
Epoch 6/25
111/111 [==============================] - 315s 3s/step - loss: 0.2746 - acc: 0.9223 - recall: 0.9189 - precision: 0.

In [ ]:
test = model.evaluate(test_generator, steps=test_stop)

16/16 [==============================] - 308s 20s/step - loss: 0.2935 - acc: 0.9475 - recall: 0.9475 - precision: 0.9475 - f1_score: 0.9460 - fbeta_score: 0.9460


In [ ]:
model.save('/content/drive/MyDrive/dataset/ENetB4new25.h5')

In [ ]:
import pandas as pd

In [ ]:
x_test = test_generator.filepaths
y_test = test_generator.labels

In [ ]:
pred = model.predict(test_generator)

16/16 [==============================] - 39s 2s/step


In [ ]:
predicted_classes = np.argmax(pred, axis = 1)
print(predicted_classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 3 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 4 1 2 2 2 3 3 3 3 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 4 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 2 2 2 2 2 2
 2 2 2 2 3 1 3 1 3 3 3 3 3 3 3 3 3 3 3 3 1 3 3 3 3 3 4 3 3 1 3 2 3 1 6 1 3
 1 3 6 3 3 3 3 3 3 3 1 3 3 3 3 1 3 1 3 3 6 3 3 2 3 1 3 1 3 3 3 3 3 3 3 3 3
 3 1 3 3 3 3 1 3 1 3 3 2 3 3 3 3 3 2 3 3 6 3 3 2 3 2 3 1 1 3 4 2 4 2 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 

In [ ]:
from sklearn.metrics import classification_report


In [ ]:
report = classification_report(y_test, predicted_classes)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       100
           1       0.84      0.97      0.90       100
           2       0.92      0.92      0.92       100
           3       0.89      0.72      0.80       100
           4       0.97      0.98      0.98       100
           5       1.00      1.00      1.00       100
           6       0.96      1.00      0.98       100
           7       1.00      0.99      0.99       100

    accuracy                           0.95       800
   macro avg       0.95      0.95      0.95       800
weighted avg       0.95      0.95      0.95       800



In [ ]:
df_marks = pd.DataFrame({'x_test': x_test,
                         'y_test': y_test,
                         'predicted_classes': predicted_classes})

In [ ]:
df_marks.to_csv('/content/drive/MyDrive/data/pred_4new25.csv')